In [103]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [104]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [105]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])


In [106]:
#Dataloader

#Path for training and testing directory
train_path='R:\cnn_test\gdriveCNN\lc_train'
test_path='R:\cnn_test\gdriveCNN\lc_test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [107]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['Benign', 'Malignant', 'Normal']


In [108]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        
        self.bn1=nn.BatchNorm2d(num_features=12)
        
        self.relu1=nn.ReLU()
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
       
        self.relu2=nn.ReLU()
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        
        self.bn3=nn.BatchNorm2d(num_features=32)
        
        self.relu3=nn.ReLU()
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
    
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
        output=self.fc(output)
            
        return output

In [109]:
model=ConvNet(num_classes=3).to(device)

In [110]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.0001,weight_decay=0.005)
loss_function=nn.CrossEntropyLoss()

In [111]:
num_epochs=40

In [112]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [113]:
print(train_count,test_count)

240 90


In [114]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print(f'Epoch: {epoch} Train Loss: {train_loss} Train Accuracy: {train_accuracy*100}%  Test Accuracy: {test_accuracy*100}%')
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: 2.23067569732666 Train Accuracy: 41.25%  Test Accuracy: 33.33333333333333%
Epoch: 1 Train Loss: 1.1690032482147217 Train Accuracy: 65.0%  Test Accuracy: 28.888888888888886%
Epoch: 2 Train Loss: 0.7508823871612549 Train Accuracy: 73.33333333333333%  Test Accuracy: 33.33333333333333%
Epoch: 3 Train Loss: 0.40847498178482056 Train Accuracy: 85.83333333333333%  Test Accuracy: 35.55555555555556%
Epoch: 4 Train Loss: 0.13030444085597992 Train Accuracy: 94.16666666666667%  Test Accuracy: 54.44444444444444%
Epoch: 5 Train Loss: 0.1376965194940567 Train Accuracy: 92.91666666666667%  Test Accuracy: 34.44444444444444%
Epoch: 6 Train Loss: 0.027869462966918945 Train Accuracy: 100.0%  Test Accuracy: 38.88888888888889%
Epoch: 7 Train Loss: 0.04843901842832565 Train Accuracy: 99.58333333333333%  Test Accuracy: 38.88888888888889%
Epoch: 8 Train Loss: 0.028773514553904533 Train Accuracy: 99.58333333333333%  Test Accuracy: 38.88888888888889%
Epoch: 9 Train Loss: 0.006120683625340462